In [ ]:
import sys

# Open the file in write mode
file = open('C6D1-RAS.txt', 'w')

# Save the current sys.stdout
original_stdout = sys.stdout

# Redirect sys.stdout to the file
sys.stdout = file

#Updated on 19 April 2023

#Importing necessary packages
import pandas as pd
import numpy as np
import time

# Initiating lists for reporting purpose

#report resource levels
report_cobots = []
report_dedicateds = []

#Utilization reports
report_cobot_travel_time = []
report_cobot_wait_time = []
report_cobot_pick_time = []

report_dedicated_travel_time = []
report_dedicated_pick_time = []

report_drop_off_station_reach_time_list = []
report_abandon_performance = []
report_abandon_triggering_events = []
report_actually_abandoned = []

report_items_in_order = []
report_aisles_in_order = []
report_cobot_incomplete_pick_time = []
report_dedicated_incomplete_pick_time = []

report_cobot_idle_time = []
report_dedicated_idle_time = []

# Capturing computational time
report_clustering_time = []
report_sequencing_time = []
report_pick_completion_trigger_time = []
report_abandon_trigger_time = []
report_return_trigger_time = []
report_limiting_alternative = []

report_custArrInst = []
# num_picked_by_cobot = []
# num_picked_by_dedicated = [] 
# aisles_by_cobot = []
# aisles_by_dedicated = []
# partial_aisles_by_cobot = []
# partial_aisles_by_dedicated = []
initialization_time = []
pick_end_time = []
total_picked_by_cobot = []
total_picked_by_dedicated = []
list_partial_picked_by_cobot = []
list_partial_picked_by_dedicated = []
total_aisles_by_cobot = []
total_aisles_by_dedicated = []
sequencing_alternative = []
abandon_alternative = []
updating_alternative = []
limiting_alternative = []

# Input Parameters -----> Need to Change for Design of Experiment
num_cob = 6          # number of deployed cobots
num_ded = 1          # number of deployed dedicated pickers
v_g = 0              # resting station aisle id
v_p = 14             # packing station aisle id
t_wave = 1800        # wave time in seconds
#num_part_cust = 150  #number of participating customers per hour
avg_pick_time = 25   #Average picking time for both in-store cusotmers and dedicated pickers
pick_time_shortening_factor = 1.0 #percent of time needed after picking the first item from a location
cobot_speed = 0.4 #cobot's speed in meter per second
dedicated_speed = 0.6 #dedicated picker's speed in meter per second

# Deciding the variation of the heuristic
# DOE CENTER #---------# DOE CENTER #---------# DOE CENTER #---------# DOE CENTER #---------# DOE CENTER #---------# DOE CENTER #---------# DOE CENTER #--------- 

#Sequencing pair - one of them needs to be zero and the other one to be 1
ranking_based_seq = 0 #no longer using it
tsp_based_seq = 0
buypicks_based_seq = 1
# Decision on whether to use abandon trigger or not
use_abandon = 1
#Decision on which alternative to use for updating decisions when there is a pick completion
use_rsia = 0 #no longer using it
use_ipinsertion = 1
#Updating decisions only when any route is infeasible
update_when_infeasible_routes = 0
report_update_when_infeasible = []

#Limiting to one or multiple insertion, 0 refers to no limit, 1 refers to limiting to only one insertion
limit_to_one_insertion = 1
no_update = 0               # No update at all


# DOE CENTER #---------# DOE CENTER #---------# DOE CENTER #---------# DOE CENTER #---------# DOE CENTER #---------# DOE CENTER #---------# DOE CENTER #---------

# Initiating the average dwell time dictionary
dict_dwell = {}
df_dwell = pd.read_excel(r'C:\Users\bhowmj\OneDrive - Rensselaer Polytechnic Institute\DQE\Data\Instacart_Aisle_buy.xlsx')
for i in range(df_dwell.shape[0]):
    dict_dwell[i+1] = df_dwell.iloc[i,5] 

# List of Participating Customers
n_part_cust_list = [100, 125, 150, 175]
report_participating_cust = []

#THE MASTER LOOP # __________#THE MASTER LOOP # __________#THE MASTER LOOP # __________#THE MASTER LOOP # __________#THE MASTER LOOP # __________ 

for part_cust_num in n_part_cust_list:     
    for order in range(0, 10):
        ord_scen = order + 1
        custArrInst = []
        for arr_count in range (0,100):
            custArrInst.append(arr_count+1)
            arr = arr_count + 1

            #For reporting purpose
            num_picked_by_cobot = []
            num_picked_by_dedicated = [] 
            aisles_by_cobot = []
            aisles_by_dedicated = []
            partial_aisles_by_cobot = []
            partial_aisles_by_dedicated = []
            cobot_travel_time = []
            cobot_wait_time = []
            cobot_pick_time = []
            dedicated_travel_time = []
            dedicated_pick_time = []
            drop_off_station_reach_time_list = [0 for i in range(num_cob+ num_ded)]
            abandon_performance = []
            invalid_abandon_trigger = []
            abandon_events = []
            cobot_incomplete_pick_time = []
            dedicated_incomplete_pick_time = []

            #Computational Time lists
            pick_completion_trigger_time = []
            abandon_trigger_time = []
            return_trigger_time = []

            #Importing the incoming order data, we need to iterate through this for different demand profile
            incoming_orders = pd.read_excel(r'C:\Users\bhowmj\OneDrive - Rensselaer Polytechnic Institute\DQE New\Data\incoming_order_arranged.xlsx',\
                                            sheet_name='order_'+str(order+1))


            # Preprocessing of incoming order data
            aisles_in_orders = incoming_orders['aisle_id'].tolist()

            aisles_and_stations = [v_g] + aisles_in_orders + [v_p]
            sink_tsp = len(aisles_and_stations) - 1
            n_picks_list = incoming_orders['number_of_picks'].tolist()
            n_picks = sum(n_picks_list)
            report_items_in_order.append(n_picks)
            report_aisles_in_order.append(len(aisles_in_orders))

            #Creating a dictionary for storing number of picks
            dict_num_picks = {}
            for i in range (len(aisles_in_orders)):
                dict_num_picks[aisles_in_orders[i]] = n_picks_list[i]


            #Importing the travel time matrix
            time_matrix = pd.read_excel(r'C:\Users\bhowmj\OneDrive - Rensselaer Polytechnic Institute\DQE\Data\Distance_Matrix.xlsx', sheet_name = 'Sheet1')
            time_mat = time_matrix.set_index('Aisle_ID')

            #Taking only the aisle_ids which are requested by the incoming orders
            time_mat_req = time_mat.loc[aisles_and_stations, aisles_and_stations]
            time_mat_wo_stations = time_mat.loc[aisles_in_orders, aisles_in_orders]

            #Transforming the time matrix according to cobot and dedicated speed
            time_mat_req_cob = time_mat_req.mul(1/cobot_speed)
            time_mat_req_ded = time_mat_req.mul(1/dedicated_speed)
            time_mat_wo_stations_cob = time_mat_wo_stations.mul(1/cobot_speed)
            time_mat_wo_stations_ded = time_mat_wo_stations.mul(1/dedicated_speed) 

            #Calculating average traveltime from each required stopping points
            avg_time_dict = {}
            avgtravelTimeList = []
            for k in range(num_cob + num_ded):
                avgtravelTimeList.append([])
                if k < num_cob:
                    for i in range(len(aisles_in_orders)):
                        avg_time_dict[aisles_in_orders[i]] = (time_mat_wo_stations_cob.iloc[:,i].sum())/(len(aisles_in_orders)-1)
                        avgtravelTimeList[k].append((time_mat_wo_stations_cob.iloc[:,i].sum())/(len(aisles_in_orders)-1))
                else:
                    for i in range(len(aisles_in_orders)):
                        avg_time_dict[aisles_in_orders[i]] = (time_mat_wo_stations_ded.iloc[:,i].sum())/(len(aisles_in_orders)-1)
                        avgtravelTimeList[k].append((time_mat_wo_stations_ded.iloc[:,i].sum())/(len(aisles_in_orders)-1))

            # Creating the dataframe and list for expected waiting time for the clustering
            df_buy_rate = pd.read_excel(r'C:\Users\bhowmj\OneDrive - Rensselaer Polytechnic Institute\DQE\Data\Instacart_Aisle_buy.xlsx')
            df_buy_rate['mtba'] = 3600/(df_buy_rate['Rev_rate']*(part_cust_num/100)) - df_buy_rate['dwell_time']
            df_buy_rate['mtba'] = np.where(df_buy_rate['mtba'] < 0, 0, df_buy_rate['mtba'])
            df_buy_rate['mtba_ded'] = 0


            #--------------------------------------------------NEED TO ADD DWELL TIME IN THE MTBA CALCULATION -----------------------------------------------
            #Creating a dictionary for buy rate
            dict_buy_rate = {}
            for i in range(df_buy_rate.shape[0]):
                dict_buy_rate[df_buy_rate.loc[i,'Insta aisle ID']] = df_buy_rate.loc[i,'Rev_rate']

            #Creating the expected waiting time list of lists for the clustering MIP 

            #First Step - Creating a list with all aisle ids
            expWaiting = []
            for i in range(num_cob):
                expWaiting.append(df_buy_rate['mtba'].tolist())
            for j in range(num_ded):
                expWaiting.append(df_buy_rate['mtba_ded'].tolist())

            #Last Step - Creating a list with the aisle ids required for this wave
            df_exp_waiting = pd.DataFrame(expWaiting)
            subset_array = np.array(aisles_in_orders)-1
            subset_list = subset_array.tolist()
            df_subset_waiting =  df_exp_waiting.loc[:,subset_list]
            expWaitList = df_subset_waiting.values.tolist()

            #Making a dictionary for expected waiting time
            dict_exp_wait_cobot = {}
            for i in aisles_in_orders:
                dict_exp_wait_cobot[i] = df_buy_rate.loc[i-1,'mtba']

            #Creating the dataframe for expected picking time 
            df_buy_rate['pick_time'] = avg_pick_time #------------> # may need to reduce to needes aisle ids
            expPickTime = [] 
            for i in range(num_cob + num_ded):
                expPickTime.append(df_buy_rate['pick_time'].tolist())

            clustering_start_time = time.time() # recording time for the clustering algorithm
            #----------------------------------------------------------------------------------------------------------------------
            # Begin: The cluster first MIP model
            #----------------------------------------------------------------------------------------------------------------------
            import gurobipy as gp
            from gurobipy import GRB
            from itertools import product

            # Declare and initialize model
            m_cluster = gp.Model('ClusterFirst')
            m_cluster.Params.LogToConsole = 0 #Do not show the optimization information in the console

            wave_time_in_seconds = t_wave
            numPickingResources = range(num_cob + num_ded)
            aisleIdsToVisit = range(len(aisles_in_orders))
            expWaiting = expWaitList
            tPick_v = expPickTime
            numPicks = n_picks_list
            t_w = wave_time_in_seconds

            t_expcomp  = []
            for k in numPickingResources:
                t_expcomp.append(m_cluster.addVar(lb = 0.0, vtype=GRB.CONTINUOUS, name =" tBeyond_k[%d]" % k))

            x = []
            for k in numPickingResources :
                x.append ([])
                for v in aisleIdsToVisit:
                    x[k].append (m_cluster.addVar(vtype=GRB.BINARY, name =" x_kv[%d ,%d]" % (k, v)))

            z_surrogate = m_cluster.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name="z_wnp" )


            #Constraint: time a picking resource goes beyond the allowed fraction of wave length for waiting and picking
            for k in numPickingResources:
                m_cluster.addConstr (t_expcomp[k] == sum(x[k][v]*expWaiting[k][v] for v in aisleIdsToVisit) + \
                                  sum(x[k][v]*tPick_v[k][v]*numPicks[v] for v in aisleIdsToVisit), "tBeyond_k [%d]" % k)


            #Constraint: each aisle id is assigned to only one picking resource
            for v in aisleIdsToVisit:
                m_cluster.addConstr (sum(x[k][v] for k in numPickingResources) == 1, "OnlyOneResource")


            #Constraint: setting an upper bound for the assigned waiting and picking time for all picking resources
            for k in numPickingResources:
                m_cluster.addConstr((z_surrogate >= t_expcomp[k] + sum(x[k][v]*avgtravelTimeList[k][v] for v in aisleIdsToVisit)), "UpBounDtBeyond[%d]"%k)


            # Defining the objective function
            m_cluster.setObjective (z_surrogate, GRB.MINIMIZE)


            # Save model
            m_cluster.write ('MultipleKnapsackProblem.lp')

            # Solve
            m_cluster.optimize ()


            time_count = []
            non_seq_index = []
            if m_cluster.status == GRB.OPTIMAL:
                print (f' optimal objective function value: {m_cluster.ObjVal}')
                for k in numPickingResources:
                    non_seq_index.append([])
                    time_count.append([])
                    for v in aisleIdsToVisit:
                        if x[k][v].X > 0.0001:
                            #print (f'picking resource {k} to visit aisle id: {v} pick and wait {expWaitList[k][v] + tPick_v[k][v]*numPicks[v] + avgtravelTimeList[k][v]}')
                            non_seq_index[k].append(v)
                            time_count[k].append(expWaitList[k][v] + tPick_v[k][v]*numPicks[v] + avgtravelTimeList[k][v])


            # Changing the assignments from index to actual aisle ids
            non_seq_assignments = []
            element = 0
            for i in non_seq_index:
                non_seq_assignments.append([])
                for j in i:
                    non_seq_assignments[element].append(aisles_in_orders[j])
                element = element + 1
            print(f'non sequenced assignment list {non_seq_assignments}')                     
            #----------------------------------------------------------------------------------------------------------------------
            # END: The cluster first MIP model
            #----------------------------------------------------------------------------------------------------------------------
            clustering_end_time = time.time() # recording time for the clustering algorithm
            clustering_time = clustering_end_time - clustering_start_time 

            sequencing_start_time = time.time()
            #----------------------------------------------------------------------------------------------------------------------
            # Start : Determining the Sequence
            #----------------------------------------------------------------------------------------------------

            # Determining the sequence of the assigned points
            #Alternative 1: Based on ranking of purchase rates
            if buypicks_based_seq == 1:
                print(non_seq_assignments)
                seq_assignments = []
                resource_count = 0
                for i in non_seq_assignments:
                    if resource_count < num_cob:
                        unranked_buy_rate_times_num_picks_list = []
                        dict_non_seq = {}
                        for j in i:
                            unranked_buy_rate_times_num_picks_list.append(dict_buy_rate[j]*dict_num_picks[j])
                            #print(f' buy rate {dict_buy_rate[j]} times num picks {dict_num_picks[j]} equals {dict_buy_rate[j]*dict_num_picks[j]}')
                        dict_non_seq = {'aisle_id': i, 'buy_rate_times_num_picks': unranked_buy_rate_times_num_picks_list}
                        df_temp_nonseq = pd.DataFrame(dict_non_seq)
                        #print(df_temp_nonseq)
                        df_temp_seq = df_temp_nonseq.sort_values(by = 'buy_rate_times_num_picks', ascending = False)
                        #print(df_temp_seq)
                        seq_assignments.append(df_temp_seq['aisle_id'].tolist())
                    resource_count = resource_count + 1

                #Adding the resting and packing station in the sequence
                for i in range (num_cob):
                    seq_assignments[i] = [v_g] + seq_assignments[i] + [v_p]
                print(f' sequenced assignments after ranking based for cobots only :{seq_assignments}')

                # The dedicated pickers always need to decide their sequence based on TSP
                from networkx import DiGraph, from_numpy_matrix, relabel_nodes, set_node_attributes
                from numpy import array
                import networkx as nx

                for i in range(num_cob + num_ded):
                #Making a list for the all the required aisle ids
                    if i >= num_cob:
                        aisles_in_route = non_seq_assignments[i]
                        route_with_stations = [v_g] + aisles_in_route + [v_p]

                        #Taking only the aisle_ids which are requested by the incoming orders
                        time_mat_req_route = time_mat.loc[route_with_stations, route_with_stations].mul(1/dedicated_speed)

                        sink_id = len(route_with_stations) - 1

                        ### Determining the TSP/VRP time for all the items by the dedicated picker(s)

                        #Setting that there is no incoming edge for the source/recharging station
                        time_mat_req_route.loc[(time_mat_req_route[0]>0), 0] = 0

                        #Setting that there is no outgoing edge for the snk/packing station
                        #packing_station_id = cobot_pickings + 1
                        time_mat_req_route.loc[v_p, :] = 0

                        # Demands (key: node, value: amount)
                        DEMAND = {}
                        for aisle in aisles_in_route:
                            DEMAND[aisle] = 1

                        # The matrix is transformed into a DiGraph
                        A = array(time_mat_req_route, dtype=[("cost", int)])
                        G = from_numpy_matrix(A, create_using=nx.DiGraph())

                        # The demands are stored as node attributes
                        set_node_attributes(G, values=DEMAND, name="demand")

                        # The depot is relabeled as Source and Sink
                        G = relabel_nodes(G, {0: "Source", sink_id: "Sink"})

                        from vrpy import VehicleRoutingProblem
                        prob = VehicleRoutingProblem(G, num_stops = 100,  load_capacity = 100 )
                        prob.num_vehicles = 1
                        prob.solve(time_limit=10) 
                        print(f'optimal route for picking resource {i} found with {prob.best_value} seconds')
                        current_optimal_route = prob.best_routes[1]
                        current_optimal_route[0] = 0
                        current_optimal_route[-1] = sink_id

                        rev_route = []
                        for i in current_optimal_route:
                            rev_route.append(route_with_stations[i])
                        seq_assignments.append(rev_route)

                print(seq_assignments)



            if ranking_based_seq == 1:
                print(non_seq_assignments)
                seq_assignments = []
                resource_count = 0
                for i in non_seq_assignments:
                    if resource_count < num_cob:
                        unranked_buy_rate_list = []
                        dict_non_seq = {}
                        for j in i:
                            unranked_buy_rate_list.append(dict_buy_rate[j])
                        dict_non_seq = {'aisle_id': i, 'buy_rate': unranked_buy_rate_list}
                        df_temp_nonseq = pd.DataFrame(dict_non_seq)
                        #print(df_temp_nonseq)
                        df_temp_seq = df_temp_nonseq.sort_values(by = 'buy_rate', ascending = False)
                        #print(df_temp_seq)
                        seq_assignments.append(df_temp_seq['aisle_id'].tolist())
                    resource_count = resource_count + 1

                #Adding the resting and packing station in the sequence
                for i in range (num_cob):
                    seq_assignments[i] = [v_g] + seq_assignments[i] + [v_p]
                print(f' sequenced assignments after ranking based for cobots only :{seq_assignments}')

                # The dedicated pickers always need to decide their sequence based on TSP
                from networkx import DiGraph, from_numpy_matrix, relabel_nodes, set_node_attributes
                from numpy import array
                import networkx as nx

                for i in range(num_cob + num_ded):
                #Making a list for the all the required aisle ids
                    if i >= num_cob:
                        aisles_in_route = non_seq_assignments[i]
                        route_with_stations = [v_g] + aisles_in_route + [v_p]

                        #Taking only the aisle_ids which are requested by the incoming orders
                        time_mat_req_route = time_mat.loc[route_with_stations, route_with_stations].mul(1/dedicated_speed)

                        sink_id = len(route_with_stations) - 1

                        ### Determining the TSP/VRP time for all the items by the dedicated picker(s)

                        #Setting that there is no incoming edge for the source/recharging station
                        time_mat_req_route.loc[(time_mat_req_route[0]>0), 0] = 0

                        #Setting that there is no outgoing edge for the snk/packing station
                        #packing_station_id = cobot_pickings + 1
                        time_mat_req_route.loc[v_p, :] = 0

                        # Demands (key: node, value: amount)
                        DEMAND = {}
                        for aisle in aisles_in_route:
                            DEMAND[aisle] = 1

                        # The matrix is transformed into a DiGraph
                        A = array(time_mat_req_route, dtype=[("cost", int)])
                        G = from_numpy_matrix(A, create_using=nx.DiGraph())

                        # The demands are stored as node attributes
                        set_node_attributes(G, values=DEMAND, name="demand")

                        # The depot is relabeled as Source and Sink
                        G = relabel_nodes(G, {0: "Source", sink_id: "Sink"})

                        from vrpy import VehicleRoutingProblem
                        prob = VehicleRoutingProblem(G, num_stops = 100,  load_capacity = 100 ) 
                        prob.num_vehicles = 1
                        prob.solve(time_limit=10) 
                        print(f'optimal route for picking resource {i} found with {prob.best_value} seconds')
                        current_optimal_route = prob.best_routes[1]
                        current_optimal_route[0] = 0
                        current_optimal_route[-1] = sink_id

                        rev_route = []
                        for i in current_optimal_route:
                            rev_route.append(route_with_stations[i])
                        seq_assignments.append(rev_route)

                print(seq_assignments)


            #Need to add tsp for the dedicateds    

            #Alternative 2: Based on TSP
            if tsp_based_seq == 1:

                from networkx import DiGraph, from_numpy_matrix, relabel_nodes, set_node_attributes
                from numpy import array
                import networkx as nx
                seq_assignments = []

                for i in range(num_cob + num_ded):
                    #Making a list for the all the required aisle ids

                    aisles_in_route = non_seq_assignments[i]
                    route_with_stations = [v_g] + aisles_in_route + [v_p]

                    #Taking only the aisle_ids which are requested by the incoming orders
                    if i < num_cob:
                        time_mat_req_route = time_mat.loc[route_with_stations, route_with_stations].mul(1/cobot_speed)
                    else: 
                        time_mat_req_route = time_mat.loc[route_with_stations, route_with_stations].mul(1/dedicated_speed)

                    sink_id = len(route_with_stations) - 1

                    ### Determining the TSP/VRP time for all the items by the dedicated picker(s)

                    #Setting that there is no incoming edge for the source/recharging station
                    time_mat_req_route.loc[(time_mat_req_route[0]>0), 0] = 0

                    #Setting that there is no outgoing edge for the snk/packing station
                    #packing_station_id = cobot_pickings + 1
                    time_mat_req_route.loc[v_p, :] = 0

                    # Demands (key: node, value: amount)
                    DEMAND = {}
                    for aisle in aisles_in_route:
                        DEMAND[aisle] = 1

                    # The matrix is transformed into a DiGraph
                    A = array(time_mat_req_route, dtype=[("cost", int)])
                    G = from_numpy_matrix(A, create_using=nx.DiGraph())

                    # The demands are stored as node attributes
                    set_node_attributes(G, values=DEMAND, name="demand")

                    # The depot is relabeled as Source and Sink
                    G = relabel_nodes(G, {0: "Source", sink_id: "Sink"})

                    from vrpy import VehicleRoutingProblem
                    prob = VehicleRoutingProblem(G, num_stops = 200,  load_capacity = 200) 
                    prob.num_vehicles = 1
                    prob.solve(time_limit=10) 
                    print(f'optimal route for picking resource {i} found with {prob.best_value} seconds')
                    current_optimal_route = prob.best_routes[1]
                    current_optimal_route[0] = 0
                    current_optimal_route[-1] = sink_id

                    rev_route = []
                    for i in current_optimal_route:
                        rev_route.append(route_with_stations[i])
                    seq_assignments.append(rev_route)

                print(seq_assignments)
            #----------------------------------------------------------------------------------------------------------------------
            # End : Determining the Sequence
            #-------------------------------------------------------------------------------------------------    
            sequencing_end_time = time.time()
            sequencing_time = sequencing_end_time - sequencing_start_time

            #----------------------------------------------------------------------------------------------------------------------
            # Start : Current Assignments List
            #-------------------------------------------------------------------------------------------------
            # Initiating Current Assignments List
            current_assignments = []
            for i in range (num_cob + num_ded):
                current_assignments.append(seq_assignments[i][1]) #taking the second element as the first one is the resting station

            #----------------------------------------------------------------------------------------------------------------------
            # End : Current Assignments List
            #-------------------------------------------------------------------------------------------------

            #----------------------------------------------------------------------------------------------------------------------
            # Start : Setting Return Triggers
            #-------------------------------------------------------------------------------------------------

            # Setting the return triggers
            trigger_return = []
            for i in range (num_cob + num_ded):
                if i < num_cob:
                    trigger_return.append(t_wave - time_mat_req_cob.loc[current_assignments[i], v_p])
                else:
                    trigger_return.append(t_wave - time_mat_req_ded.loc[current_assignments[i], v_p])
            print(f' wave time trigger list {trigger_return}')

            #----------------------------------------------------------------------------------------------------------------------
            # End: Setting Return Triggers
            #-------------------------------------------------------------------------------------------------


            #Initiating the reach time list
            system_time = 0 # as this is the initial trigger
            # Determinng the reach time list
            reach_time_list = []
            for i in range (num_cob + num_ded):
                if i < num_cob:
                    reach_time_list.append(system_time + time_mat_req_cob.loc[v_g,current_assignments[i]])
                    cobot_travel_time.append(time_mat_req_cob.loc[v_g,current_assignments[i]])
                else:
                    reach_time_list.append(system_time + time_mat_req_ded.loc[v_g,current_assignments[i]])
                    dedicated_travel_time.append(time_mat_req_ded.loc[v_g,current_assignments[i]])
            print(reach_time_list)

            #----------------------------------------------------------------------------------------------------------------------
            # Start: Defining and Setting Abandon Triggers
            #----------------------------------------------------------------------------------------------------------------------
            if use_abandon == 1:

                def abaondon_policy (sequence_with_stations, dict_buy_rate, n_picks_list, system_time, time_mat_req, dict_exp_wait_cobot):
                    from scipy.optimize import minimize
                    from scipy.stats import poisson

                    # Analytical Approach for the initial solution:
                    seq_wo_stations = sequence_with_stations[1:-1]
                    buy_rate_list_abandon = []
                    items_to_pick_list_abandon = []
                    for i in seq_wo_stations:
                        buy_rate_list_abandon.append(dict_buy_rate[i])
                        items_to_pick_list_abandon.append(dict_num_picks[i])

                    #Creating the buy rate times items to pick product and their list
                    prod_sum = 0
                    prod_list = []

                    for i in range(len(buy_rate_list_abandon)):
                            prod_sum = prod_sum + buy_rate_list_abandon[i]*items_to_pick_list_abandon[i]
                            prod_list.append(buy_rate_list_abandon[i]*items_to_pick_list_abandon[i])
                    print(prod_sum)
                    print(prod_list)

                    #Calculating time allowed for the rest of the wave
                    total_travel_abandon = []
                    for i in range(len(sequence_with_stations)-1):
                        total_travel_abandon.append(time_mat_req.loc[sequence_with_stations[i], sequence_with_stations[i+1]])

                    total_exp_pick_abandon = []
                    for i in seq_wo_stations:
                        total_exp_pick_abandon.append(dict_exp_wait_cobot[i])

                    time_allowed = t_wave - system_time #- sum(total_travel_abandon) - sum(total_exp_pick_abandon)

                    ratio_array =  np.array(prod_list)/prod_sum
                    time_to_wait = ratio_array*time_allowed
                    print(time_to_wait)


                    #Defining the objective function of the 
                    def objective(x):
                        def to_calc(j0):
                            return (1 - poisson.cdf(k=0, mu=buy_rate_list_abandon[j0]*1.5*(x[j0]/3600)))*items_to_pick_list_abandon[j0]
                        sum_to_do = 0
                        for i in range(len(x)):
                            sum_to_do = sum_to_do + to_calc(i)
                        return -sum_to_do


                    def constraint1(x):
                        list1 = [x[i] for i in range(len(x))]
                        return - sum(list1) + time_allowed

                    x0 = time_to_wait
                    print('Objective function with Initial Guess: ' + str(objective(x0)))

                    # optimize
                    bnds = [(0, t_wave) for i in range(len(seq_wo_stations))]
                    con1 = {'type': 'ineq', 'fun': constraint1}
                    # con2 = {'type': 'eq', 'fun': constraint2}
                    #cons = ([con1,con2])
                    cons = ([con1])
                    solution = minimize(objective,x0,method='SLSQP', bounds=bnds,constraints=cons)
                    x = solution.x

                    # show final objective
                    print('Final SLSQP Objective: ' + str(objective(x)))

                    # print solution
                    print('Solution')
                    print(f'optimal gap {objective(x0)/objective(x)}')
                    return x[0]
                    del x

                #Setting the initial abandon triggers
                trigger_abandon = []
                for i in range (num_cob):
                    maxwait = abaondon_policy (seq_assignments[i], dict_buy_rate, n_picks_list, system_time, time_mat_req_cob, dict_exp_wait_cobot)
                    trigger_abandon.append(reach_time_list[i] + maxwait)
                for i in range (num_ded):
                    trigger_abandon.append(t_wave)
                print(f'Initial abandon triggers set at :{trigger_abandon}')
            #Setting the trigger return list to an empty one if we decide not to use the abandon trigger
            if use_abandon == 0:
                trigger_abandon = [t_wave for i in range (num_cob + num_ded)]
            #----------------------------------------------------------------------------------------------------------------------
            # End: Defining and Setting Abandon Triggers
            #----------------------------------------------------------------------------------------------------------------------
            #----------------------------------------------------------------------------------------------------------------------
            # Start: Initiating Unvisited Points List
            #----------------------------------------------------------------------------------------------------------------------
            unvisited_list = aisles_in_orders[:]
            print(f'unvisited points list {unvisited_list}')
            #----------------------------------------------------------------------------------------------------------------------
            # End: Initiating Unvisited Points List
            #----------------------------------------------------------------------------------------------------------------------

            #----------------------------------------------------------------------------------------------------------------------
            # Start: Determining the initial trigger
            #----------------------------------------------------------------------------------------------------------------------

    #         # Determinng the reach time list
    #         reach_time_list = []
    #         for i in range (num_cob + num_ded):
    #             reach_time_list.append(system_time + time_mat_req.loc[v_g,current_assignments[i]])
    #         print(reach_time_list)

            #Determinng the initial sync time list
            sync_time = []
    #         arr_mat = pd.read_excel(r'C:\Users\horneh\OneDrive - Rensselaer Polytechnic Institute\DQE New\Data\arr_'\
    #                                 + str(arr)+ '.xlsx', sheet_name = 'arr_' + str(arr)).set_index('arrivals')
            arr_mat = pd.read_excel(r'C:\Users\bhowmj\OneDrive - Rensselaer Polytechnic Institute\DQE New\Data\IP Fixed Results\Arrivals'\
                                    + str(part_cust_num) +'.xlsx',sheet_name = 'arr_'+str(arr)).set_index('arrivals')


            for i in range (num_cob + num_ded):
                aisle_id = current_assignments[i]
                print(f'considering aisle id: {aisle_id}')
                reach_time = reach_time_list[i]
                print(f'reach time: {reach_time}')
                if i<num_cob:
                    for j in range(arr_mat.shape[0]):
                        if arr_mat.iloc[j, aisle_id -1] + dict_dwell[aisle_id] >= reach_time:
                            if reach_time <= arr_mat.iloc[j, aisle_id -1]:
                                sync_time.append(arr_mat.iloc[j, aisle_id -1])
                            else:
                                sync_time.append(reach_time)
                            break
                else:
                    sync_time.append(reach_time) 
            print(f'initial sync time list: {sync_time}')

            # Determining the initial pick completion time
            pick_completion_trigger = []
            for i in range(num_cob + num_ded):
                pick_time  = avg_pick_time + (dict_num_picks[current_assignments[i]]-1)*pick_time_shortening_factor*avg_pick_time
                pick_completion_trigger.append(sync_time[i]+pick_time)
                print(f'pick completes at {sync_time[i]+pick_time} at point {current_assignments[i]} with # picks {dict_num_picks[current_assignments[i]]}')

            #Determining the next trigger:
            def next_trigger (pick_completion_trigger, trigger_return, trigger_abandon):
                trig_list = pick_completion_trigger + trigger_return + trigger_abandon
                next_trig_time = min(trig_list)
                next_trig_index = trig_list.index(next_trig_time)
                if next_trig_index < num_cob + num_ded:
                    next_trig_type = 'pickCompletion'
                elif next_trig_index < 2*(num_cob + num_ded) and next_trig_index >= num_cob + num_ded:
                    next_trig_type = 'return'
                else:
                    next_trig_type = 'abandon'
                next_trig_resource = (next_trig_index) % (num_cob + num_ded)
                nt_point = current_assignments[next_trig_resource]
                return next_trig_time, next_trig_type, next_trig_resource, nt_point  


            try: 
                nt_time, nt_type, nt_resource, nt_point = next_trigger (pick_completion_trigger, trigger_return, trigger_abandon) #Here nt refers to next trigger
                print(f' first trigger will happen at {nt_time} seconds with trigger type {nt_type} by resource {nt_resource} at point {nt_point}') 
            except:
                trigger_return_backup = []
                for series in trigger_return:
                    series_values = list(series.values)
                    trigger_return_backup.append(series_values[0])
                trigger_return = trigger_return_backup[:]
                
                print(f'updated wave time trigger: {trigger_return}')
                
                nt_time, nt_type, nt_resource, nt_point = next_trigger (pick_completion_trigger, trigger_return, trigger_abandon) #Here nt refers to next trigger
                print(f' first trigger will happen at {nt_time} seconds with trigger type {nt_type} by resource {nt_resource} at point {nt_point}') 

                
            #----------------------------------------------------------------------------------------------------------------------
            # End: Determining the initial trigger
            #----------------------------------------------------------------------------------------------------------------------
            print('---------------END INITIALIZATION---------------END INITIALIZATION---------------END INITIALIZATION---------------END INITIALIZATION')
            # Initiate lists to identify performance difference between rsia and ip
            range_exp_pick_comp_time = []
            max_exp_pick_comp_time = []
            num_insertions_performed = []
            insertion_improvement = []
            #----------------------------------------------------------------------------------------------------------------------
            # Start: Updating Decisions After Every trigger
            #----------------------------------------------------------------------------------------------------------------------
            #Setting the termination criteria
            termination_crtieria = [v_p for i in range(num_cob + num_ded)]
            system_time = nt_time

            #Making a list of sequence without the stations
            seq_wo_stations = []
            for i in range (num_cob + num_ded):
                seq_wo_stations.append(seq_assignments[i][1:-1])

            trig_num = 0 # to keep track of the number of triggers happening

            #THE UPDATING LOOP --THE UPDATING LOOP --THE UPDATING LOOP --THE UPDATING LOOP --THE UPDATING LOOP --THE UPDATING LOOP --THE UPDATING LOOP --THE UPDATING LOOP --
            while current_assignments != termination_crtieria:

                    #Updating decisions if the trigger is of pick completion type    
                if nt_type == 'pickCompletion':
                    #Record the pick for reporting purpose
                    if num_cob == 0:
                        aisles_by_dedicated.append(nt_point)
                        num_picked_by_dedicated.append(dict_num_picks[nt_point])
                        print(f'# of items successfully picked by dedicated {dict_num_picks[nt_point]}')
                        dedicated_pick_time.append(dict_num_picks[nt_point]*avg_pick_time)
                    
                    
                    elif nt_resource < num_cob:
                        aisles_by_cobot.append(nt_point)
                        num_picked_by_cobot.append(dict_num_picks[nt_point])
                        print(f'# of items successfully picked by cobot {dict_num_picks[nt_point]}')
                        cobot_pick_time.append(dict_num_picks[nt_point]*avg_pick_time)
                        cobot_wait_time.append(nt_time - reach_time_list[nt_resource] - dict_num_picks[nt_point]*avg_pick_time)

                    else:
                        aisles_by_dedicated.append(nt_point)
                        num_picked_by_dedicated.append(dict_num_picks[nt_point])
                        print(f'# of items successfully picked by dedicated {dict_num_picks[nt_point]}')
                        dedicated_pick_time.append(dict_num_picks[nt_point]*avg_pick_time)

                    seq_wo_stations = []
                    for i in range (num_cob + num_ded):
                        seq_wo_stations.append(seq_assignments[i][1:-1])

                    print(f'removing {nt_point} from the unvisited list')
                    unvisited_list.remove(nt_point)
                    #seq_assignments[nt_resource].remove()
                    if len(unvisited_list) == 0:
                        current_assignments[nt_resource] = v_p
                        if nt_resource < num_cob:
                            cobot_travel_time.append(time_mat_req_cob.loc[nt_point,v_p])
                            drop_off_station_reach_time_list[nt_resource] = nt_time + time_mat_req_cob.loc[nt_point,v_p]
                        else:
                            dedicated_travel_time.append(time_mat_req_ded.loc[nt_point,v_p])
                            drop_off_station_reach_time_list[nt_resource] = nt_time + time_mat_req_ded.loc[nt_point,v_p]
                        reach_time_list[nt_resource] = t_wave
                        pick_completion_trigger[nt_resource] = t_wave
                        trigger_return[nt_resource] = t_wave
                        trigger_abandon[nt_resource] = t_wave
                        print('there is no more unvisited point, so we send the resource to packing station')

                    elif len(unvisited_list)>0:
                        #Set a timestamp for calculating the updating time
                        pick_completion_start_time = time.time()

                        #Calculate expected pick completion time
                        exp_pick_comp_time = []
                        for i in range(num_cob + num_ded):
                            if i < num_cob:
                                exp_wait_time_trig = [dict_exp_wait_cobot[j] for j in seq_wo_stations[i]]
                            else:
                                exp_wait_time_trig = [0 for j in seq_wo_stations[i]]
                            exp_pick_time_trig = [avg_pick_time + (dict_num_picks[j]-1)*pick_time_shortening_factor*avg_pick_time for j in seq_wo_stations[i]]
                            travel_time_trig = []
                            seq_for_travel_time_calc = seq_wo_stations[i]+[v_p]
                            for k in range(len(seq_for_travel_time_calc)-1):
                                if i <num_cob:
                                    travel_time_trig.append(time_mat_req_cob.loc[seq_for_travel_time_calc[k],seq_for_travel_time_calc[k+1]])
                                else:
                                    travel_time_trig.append(time_mat_req_ded.loc[seq_for_travel_time_calc[k],seq_for_travel_time_calc[k+1]])

                            if current_assignments[i] == v_p:
                                exp_pick_comp_time.append(reach_time_list[i])

                            elif nt_time < sync_time[i]:
            #                 if nt_time < reach_time_list[i] or (nt_time > reach_time_list[i] and nt_time < sync_time[i]):
                                #print(f'trigger time {nt_time} of resource {nt_resource} is earlier than reach time of resource {i} or earlier than sync time of {sync_time[i]}')
                                exp_pick_comp_time.append(sum(exp_wait_time_trig) + sum(exp_pick_time_trig) + sum(travel_time_trig) + reach_time_list[i])

                            elif nt_time >= sync_time[i] and nt_time < pick_completion_trigger[i]:          
                                #print(f'trigger time: {nt_time}s of resource {nt_resource} is later than sync time of {sync_time[i]} of resource {i}')
                                exp_pick_comp_time.append(sum(exp_wait_time_trig) + sum(exp_pick_time_trig) + sum(travel_time_trig) \
                                                          + nt_time - exp_wait_time_trig[0])

                            elif nt_time == pick_completion_trigger[i]:
                                #print(f'pick completion calculation for the triggered resource # {nt_resource}')
                                exp_pick_comp_time.append(sum(exp_wait_time_trig) + sum(exp_pick_time_trig) + sum(travel_time_trig) \
                                                          + nt_time - exp_wait_time_trig[0] - exp_pick_time_trig[0])

                        print(f'expected pick completion time list {exp_pick_comp_time}')

                        # Storing information for comparison with rsia vs ip
                        range_exp_pick_comp_time.append(max(exp_pick_comp_time)- min(exp_pick_comp_time))
                        max_exp_pick_comp_time.append(max(exp_pick_comp_time)) 

                        #Creating a condition to find if we do not have any infeasible expected pick completion time                   
                        greater_than_wave = 1 # a dummy setup to run the updating of sequences always
                        if update_when_infeasible_routes == 1:
                            greater_than_wave = 0
                            for i in range(num_cob + num_ded):
                                if exp_pick_comp_time[i]>= t_wave:
                                    greater_than_wave = 1
                        elif no_update == 1: # Condition to stop any update after initialization
                            greater_than_wave = 0 

                        #Creating a new set to store the sequence without the current assignments as current assignments cannot be moved
                        seq_wo_current_assignments = seq_wo_stations[:] 
                        for i in range(num_cob + num_ded):
                            if current_assignments[i] == v_p:
                                seq_wo_current_assignments[i] = []
                            else:
                                seq_wo_current_assignments[i].remove(current_assignments[i])
    #                             seq_wo_current_assignments[i].remove(seq_wo_current_assignments[i][0])

                        print(f'sequence without current assignments: {seq_wo_current_assignments}')

                        #Creating a new list of lists with the current assignments
                        seq_with_current_assignments = []
                        for i in range(num_cob + num_ded):
                            curr_assignment_index = seq_assignments[i].index(current_assignments[i])
                            seq_with_current_assignments.append(seq_assignments[i][curr_assignment_index:])
                        print(f'sequence with current assignments: {seq_with_current_assignments}')

                        #Calculate slack time for each resources
                        not_candidate = [] # this list will contain the set of resources who are assigned to go to the packing station
                        for i in range(num_cob + num_ded):
                            if current_assignments[i] == v_p:
                                not_candidate.append(i)
            #                 elif len(seq_with_current_assignments[i]
                        candidate_resources = list(set([i for i in range(num_cob + num_ded)]) - set(not_candidate))
                        print(f'candidate resources {candidate_resources}')
                        if len(not_candidate) == 0:         
                            slack_time = [min(t_wave, max(exp_pick_comp_time)) - exp_pick_comp_time[i] for i in range(num_cob + num_ded)]
                        else:
                            shortened_exp_pick_comp_time = [exp_pick_comp_time[i] for i in candidate_resources]
                            print(f'shortened exp pick comp time {shortened_exp_pick_comp_time}')
                            for i in range(num_cob+num_ded):
                                if i not in candidate_resources:
                                    exp_pick_comp_time[i] = max(shortened_exp_pick_comp_time) - 1
                            slack_time = [min(t_wave, max(exp_pick_comp_time)) - exp_pick_comp_time[i] for i in range(num_cob + num_ded)]


                        print(f'slack time: {slack_time}')

                        #Revised slack time list for the IP insertion, need positive values
                        slack_time_rev = []
                        rev_exp_pick_comp_time = [exp_pick_comp_time[i] for i in candidate_resources]
                        for i in range(num_cob + num_ded):
                            if i in candidate_resources:
                                slack_time_rev.append(max(rev_exp_pick_comp_time) - exp_pick_comp_time[i])
                            else:
                                slack_time_rev.append(0)
                        print(f'revised slack time: {slack_time_rev}')


                        #Identify K^move, the set of resources considered for a stopping point removal
                        num_exp_pick_comp_over_t_wave = 0
                        for i in range(num_cob+num_ded):
                            if exp_pick_comp_time[i] > t_wave:
                                num_exp_pick_comp_over_t_wave = num_exp_pick_comp_over_t_wave + 1

                        K_move = []
                        K_move_candidates = candidate_resources[:]
                        print(f'candidates for k move {K_move_candidates}')

                        if num_exp_pick_comp_over_t_wave == num_cob + num_ded:
                            print('all the resources have expected pick comp time over the wave')
                            K_move.append(exp_pick_comp_time.index(max(exp_pick_comp_time)))
                        else:
                            for i in range(num_cob + num_ded):                            
                                if exp_pick_comp_time[i]> t_wave and i in candidate_resources and len(seq_with_current_assignments[i])>2:
                                    K_move.append(i)
                        if len(K_move) == 0:
                            print(f'no exp val is over the wave')
                            if len(seq_with_current_assignments[exp_pick_comp_time.index(max(exp_pick_comp_time))])>2:
                                print(f'here')
                                K_move.append(exp_pick_comp_time.index(max(exp_pick_comp_time)))
                            else: #this is required for the case when the max exp value is not a candidate resource
                                print('here 2')
                                K_move_candidates.remove(exp_pick_comp_time.index(max(exp_pick_comp_time)))
                                if len(K_move_candidates) !=0:
                                    K_move.append(exp_pick_comp_time.index(max([exp_pick_comp_time[i] for i in K_move_candidates]))) 

                        print(f'K_move: {K_move}')

                        K_hold = []
                        for i in range(num_cob + num_ded):
                            if i not in K_move and i in candidate_resources :
                                K_hold.append(i)
                        print(f'K_hold:{K_hold}')

                        # Identify V^move, the shopping points that are being considered for removal from their current sequence
                        V_move = []
                        dict_time_reduction_V_move = {}
                        for i in range(num_cob + num_ded):
                            if i in K_move:
                                V_move = V_move + seq_wo_current_assignments[i]
                                for j in range (len(seq_with_current_assignments[i])-2):
                                    pick_time_reduction = avg_pick_time + (dict_num_picks[seq_with_current_assignments[i][j+1]]-1)\
                                    *pick_time_shortening_factor*avg_pick_time

                                    if i<num_cob:
                                        travel_time_reduction = time_mat_req_cob.loc[seq_with_current_assignments[i][j],seq_with_current_assignments[i][j+1]] +\
                                        time_mat_req_cob.loc[seq_with_current_assignments[i][j+1],seq_with_current_assignments[i][j+2]]-\
                                        time_mat_req_cob.loc[seq_with_current_assignments[i][j],seq_with_current_assignments[i][j+2]]
                                    else:
                                        travel_time_reduction = time_mat_req_ded.loc[seq_with_current_assignments[i][j],seq_with_current_assignments[i][j+1]] +\
                                        time_mat_req_ded.loc[seq_with_current_assignments[i][j+1],seq_with_current_assignments[i][j+2]]-\
                                        time_mat_req_ded.loc[seq_with_current_assignments[i][j],seq_with_current_assignments[i][j+2]]


                                    if i<num_cob:
                                        wait_time_reduction = dict_exp_wait_cobot[seq_with_current_assignments[i][j+1]]
                                    else:
                                        wait_time_reduction = 0
                                    dict_time_reduction_V_move[seq_with_current_assignments[i][j+1]] = pick_time_reduction + travel_time_reduction + wait_time_reduction

                        print(f' V_move :{V_move}') 
                        print(f' dict_time_reduction_V_move: {dict_time_reduction_V_move}')

                        if len(V_move) > len(dict_time_reduction_V_move):
                            additional_element = list(set(V_move) - set(list(dict_time_reduction_V_move.keys())))
                            print(f' found {additional_element} not present in the V-move time reduction dictionary')
                            V_move = list(set(V_move) - set(additional_element))

                        #Condition if there is no V-move points
            #             if len(V_move) != 0:

                        # Identify V^hold, the shopping points that are being considered for removal from their current sequence
                        V_hold = []
                        for i in range(num_cob + num_ded):
                            if i in K_hold:
                                V_hold = V_hold + seq_wo_stations[i]
                        print(f'V_hold :{V_hold}') 


                        # Define the set G where the elements are the detaour positions in the K^hold sequences
                        detour_positions = []

                        for i in range(num_cob + num_ded):
                            if i in K_hold:
                                detour_positions.append(len(seq_with_current_assignments[i]) -1)
                                print(f'{len(seq_with_current_assignments[i]) -1} detour positions found in resource {i}')
                            else:
                                detour_positions.append(0)
                        num_detour_positions = sum(detour_positions)
                        print(f'total detour positions found: {num_detour_positions}')

                        #----------------------------------------------------------------------------------------------------------------------------------------------------------
                        # Start: Recursive Single Insertion Algorithm
                        #----------------------------------------------------------------------------------------------------------------------------------------------------------
                        if use_rsia == 1 and greater_than_wave==1:
                            start_rsia = time.time()

                            print('-------------------------------------------START: RECURSIVE SINGLE INSERTION ALGORITHM ----------------------------------------------------------')
                            max_expected_pick_comp_time_improvement = []
                            potential_exp_pick_comp_time = []
                            additional_exp_pick_comp_time = []
                            best_v_move = 0
                            best_k_hold = 0
                            best_detour_position = 0
                            current_best_improvement = 0

                            #Calculating the time that can be reduced after removing a stopping point from each sequence of K_move   

                            for i in V_move:
                                add_exp_pick = avg_pick_time + (dict_num_picks[i]-1)*pick_time_shortening_factor*avg_pick_time
                                time_reduction_this_point = dict_time_reduction_V_move[i]
                                #print(f'additional exp pick time {add_exp_pick}')
                                for j in K_hold:
                                    if j<num_cob:
                                        add_exp_wait = dict_exp_wait_cobot[i]
                                    else:
                                        add_exp_wait = 0
                                    #print(f'additional exp wait time {add_exp_wait}')
                                    for k in range (len(seq_with_current_assignments[j])-1):
                                        if j < num_cob:
                                            add_exp_travel = time_mat_req_cob.loc[seq_with_current_assignments[j][k],i] + time_mat_req_cob.loc[i, seq_with_current_assignments[j][k+1]] -\
                                            time_mat_req_cob.loc[seq_with_current_assignments[j][k], seq_with_current_assignments[j][k+1]]
                                        else:
                                            add_exp_travel = time_mat_req_ded.loc[seq_with_current_assignments[j][k],i] + time_mat_req_ded.loc[i, seq_with_current_assignments[j][k+1]] -\
                                            time_mat_req_ded.loc[seq_with_current_assignments[j][k], seq_with_current_assignments[j][k+1]]

                                        #print(f'additional exp travel time {add_exp_travel}')
                                        total_exp_time_add = add_exp_travel + add_exp_wait + add_exp_pick
                                        #print(f'{total_exp_time_add}s total expected picking time to add with components {add_exp_travel}, {add_exp_wait} ,and {add_exp_pick}')
                                        temp_exp_pick_comp_time = exp_pick_comp_time[:]
                                        temp_exp_pick_comp_time[j] = exp_pick_comp_time[j] + total_exp_time_add
                                        #print(f'{temp_exp_pick_comp_time[j]}s updated exp pick comp time to add V_move:{i}, in sequence of K_hold:{j}, and at position{k}')
                                        additional_exp_pick_comp_time.append(temp_exp_pick_comp_time[j])

                                        #reducing the expected pick completion time from the specific resource's expected pick completion time
                                        for km in K_move:
                                            if i in seq_wo_current_assignments[km]:
                                                temp_exp_pick_comp_time[km] = temp_exp_pick_comp_time[km] - time_reduction_this_point

                                        #print(f'current improvement of max exp pick comp time {max(exp_pick_comp_time) - max(temp_exp_pick_comp_time)}')
                                        if current_best_improvement == (max(exp_pick_comp_time) - max(temp_exp_pick_comp_time)):
                                            try: #condition if there is no best_seq_exp_pick_comp_time list
                                                if sum(best_seq_exp_pick_comp_time)>sum(temp_exp_pick_comp_time):
                                                    print(f'although equal improvement, but sum is lower')
                                                    print(f'current bext seq:{best_seq_exp_pick_comp_time}')
                                                    best_seq_exp_pick_comp_time = temp_exp_pick_comp_time[:]
                                                    best_v_move = i
                                                    best_k_hold = j
                                                    best_detour_position = k
                                                    current_best_improvement = max(exp_pick_comp_time) - max(temp_exp_pick_comp_time)
                                            except: 
                                                print(f'no improved sequence found yet')

                                        elif current_best_improvement < max(exp_pick_comp_time) - max(temp_exp_pick_comp_time):
                                            print(f'current best improvement of {max(exp_pick_comp_time) - max(temp_exp_pick_comp_time)}s')
                                            best_seq_exp_pick_comp_time = temp_exp_pick_comp_time[:]
                                            best_v_move = i
                                            best_k_hold = j
                                            best_detour_position = k
                                            current_best_improvement = max(exp_pick_comp_time) - max(temp_exp_pick_comp_time)
                            end_rsia = time.time()
                            print(f'RSIA computational time {end_rsia - start_rsia}')

                            #Condition if there is no improvement
                            no_change_rsia=0
                            if best_v_move == 0 and best_k_hold == 0 and best_detour_position == 0:
                                print('no improvements found after RSIA')
                                no_change_rsia = 1
                                num_insertions_performed.append(0)
                                insertion_improvement.append(0)

                            if no_change_rsia == 0:
                                num_insertions_performed.append(1)
                                insertion_improvement.append(current_best_improvement)

                            print(f'decision is to insert {best_v_move} in the sequence of {best_k_hold} at detour position \
                            {best_detour_position} with best improvement of {current_best_improvement}')
                            print(f'current sequence with current assignments: {seq_with_current_assignments}')
                            post_decision_seq_assignments =  seq_with_current_assignments[:]
                            if no_change_rsia==0:
                                post_decision_seq_assignments[best_k_hold].insert(best_detour_position+1,best_v_move)
                                for i in K_move:
                                    if best_v_move in seq_with_current_assignments[i]:
                                        post_decision_seq_assignments[i].remove(best_v_move)
                            print(f'post decision assignments {post_decision_seq_assignments}')

                            # Need to structure the the post decision seq assignments with the complete sequence
                            for i in range (num_cob + num_ded):
                                for j in range(len(seq_assignments[i])):
                                    if seq_assignments[i][j] == current_assignments[i]:
                                        break
                                    if seq_assignments[i][j] != post_decision_seq_assignments[i][j]:
                                        post_decision_seq_assignments[i].insert(j,seq_assignments[i][j])

                            # Remove the already picked point to finalize the resource sequences
                            prev_point_of_picked_point_index = post_decision_seq_assignments[nt_resource].index(nt_point) - 1
                            post_decision_seq_assignments[nt_resource].remove(post_decision_seq_assignments[nt_resource][prev_point_of_picked_point_index])
                            print(f'removed {nt_point} from resource {nt_resource} as all items have been picked')
                            seq_assignments = post_decision_seq_assignments[:]        
                            print(f' finalized post decision sequence after RSIA and sequence adjustments{seq_assignments}')

                        #-----------------------------------------------------------------------------------------------------------------------------------------------------------
                        # End: Recursive Single Insertion Algorithm
                        #-----------------------------------------------------------------------------------------------------------------------------------------------------------

                        #-----------------------------------------------------------------------------------------------------------------------------------------------------------
                        # START: IP Insertion 
                        #-----------------------------------------------------------------------------------------------------------------------------------------------------------


                        if use_ipinsertion == 1 and greater_than_wave==1:
                            print(f'------------------------------------- START: IP INSERTION -------------------------------------------')
                            print(f'seq assignments {seq_assignments}')
                            #Determine the input variables
                            t_add = []
                            t_deduct = []
                            y_insertion = []
                            # for i in range(len(K_hold)):
                            #     y_insertion.append([])
                            V_move_counter = 0

                            seq_with_current_assignments = []
                            for i in range(num_cob + num_ded):
                                curr_assignment_index = seq_assignments[i].index(current_assignments[i])
                                seq_with_current_assignments.append(seq_assignments[i][curr_assignment_index:])
                            print(f'sequence with current assignments: {seq_with_current_assignments}')


                            #Create the input list for time deductible
                            for k in range(num_cob+num_ded):
                                t_deduct.append([])
                                if k in K_move:
                                    for v1 in V_move:
                                        if v1 in seq_assignments[k]:
                                            t_deduct[k].append(dict_time_reduction_V_move[v1])
                                        else:
                                            t_deduct[k].append(0)
                                elif k in K_hold:
                                    for v1 in V_move:
                                        t_deduct[k].append(0)
                                else: #this is required when one resource's current assignment is set for the packing station and is not a part of either K_move or K_hold
                                    for v1 in V_move:
                                        t_deduct[k].append(0)


                            # Create the table with binary value indicating if a detour position is within a resource's sequence            
                            y_insertion = []
                            for k in range(num_cob + num_ded):
                                y_insertion.append([])
                                if k in K_hold:
                                    for j in K_hold:
                                        if j==k:
                                            for i in range(len(seq_with_current_assignments[j])-1):
                                                y_insertion[k].append(1)
                                        else:
                                            for i in range(len(seq_with_current_assignments[j])-1):
                                                y_insertion[k].append(0)
                                else:
                                    for i in range (num_detour_positions):
                                        y_insertion[k].append(0)       

                            for i in V_move:
                                t_add.append([])
                                add_exp_pick = avg_pick_time + (dict_num_picks[i]-1)*pick_time_shortening_factor*avg_pick_time
                            #     time_reduction_this_point = dict_time_reduction_V_move[i]

                                #print(f'additional exp pick time {add_exp_pick}')
                                for j in K_hold:
                                    if j<num_cob:
                                        add_exp_wait = dict_exp_wait_cobot[i]
                                    else:
                                        add_exp_wait = 0
                            #         y_insertion.append([])


                                    #print(f'additional exp wait time {add_exp_wait}')
                                    if j < num_cob:
                                        for k in range (len(seq_with_current_assignments[j])-1):
                                            add_exp_travel = time_mat_req_cob.loc[seq_with_current_assignments[j][k],i] + time_mat_req_cob.loc[i, seq_with_current_assignments[j][k+1]] -\
                                            time_mat_req_cob.loc[seq_with_current_assignments[j][k], seq_with_current_assignments[j][k+1]]
                                            #print(f'additional exp travel time {add_exp_travel}')
                                            total_exp_time_add = add_exp_travel + add_exp_wait + add_exp_pick
                                            t_add[V_move_counter].append(total_exp_time_add)
                                    else:
                                        for k in range (len(seq_with_current_assignments[j])-1):
                                            add_exp_travel = time_mat_req_ded.loc[seq_with_current_assignments[j][k],i] + time_mat_req_ded.loc[i, seq_with_current_assignments[j][k+1]] -\
                                            time_mat_req_ded.loc[seq_with_current_assignments[j][k], seq_with_current_assignments[j][k+1]]
                                            #print(f'additional exp travel time {add_exp_travel}')
                                            total_exp_time_add = add_exp_travel + add_exp_wait + add_exp_pick
                                            t_add[V_move_counter].append(total_exp_time_add)

                                V_move_counter = V_move_counter + 1

                            print(f'time additional lists {t_add}')
                            print(f'time deduction lists {t_deduct}')
                            print(f' y insertion list {y_insertion}')

                            if not any(t_add):
                                run_ipinsertion = 0
                                print("Cannot run the IP insertion.")
                            else:
                                run_ipinsertion = 1
                                print("Next Step - Run the IP Insertion.")


                            if run_ipinsertion == 1:
                                # THE IP INSERTION MODEL ------------THE IP INSERTION MODEL ------------THE IP INSERTION MODEL ------------THE IP INSERTION MODEL ------------
                                import gurobipy as gp
                                from gurobipy import GRB
                                from itertools import product

                                #Declare the model for the insertion IP
                                m_insertion = gp.Model()
                                m_insertion.ModelSense = GRB.MINIMIZE
                                m_insertion.Params.LogToConsole = 0

                                #Declare ranges
                                v_move_range = range(len(V_move))
                                detour_pos_range = range(len(t_add[0])) 

                                ### DECISION VARIABLES IP INSERTION ###### DECISION VARIABLES IP INSERTION ###### DECISION VARIABLES IP INSERTION ###### DECISION VARIABLES IP INSERTION ###
                                #Declare decision variable xij
                                x_insertion = []
                                for i in v_move_range:
                                    x_insertion.append([])
                                    for j in detour_pos_range:
                                        x_insertion[i].append(m_insertion.addVar(vtype = GRB.BINARY))

                                # Declare decision variable t_expupdated 
                                t_expupdated = []
                                for i in range (num_cob+num_ded):
                                    t_expupdated.append(m_insertion.addVar(lb=0.0, vtype=GRB.CONTINUOUS))

                                # Declare decision variable z_upbound
                                z_upbound = m_insertion.addVar(lb=0.0, vtype=GRB.CONTINUOUS)

    #                             #Setting the objective function
    #                             m_insertion.setObjective((z_upbound + 0.01*sum(t_expupdated[i] for i in range(num_cob + num_ded))) , GRB.MINIMIZE)

                                #Primary Objective
                                m_insertion.setObjectiveN(z_upbound, index=0, priority =2, abstol = 0.00)

                                #Secondary Objective
                                m_insertion.setObjectiveN(gp.quicksum(t_expupdated[i] for i in range(num_cob + num_ded)), index=1, priority =1)

                                ### CONSTRAINTS IP INSERTION ###### CONSTRAINTS IP INSERTION ###### CONSTRAINTS IP INSERTION ###### CONSTRAINTS IP INSERTION ###### CONSTRAINTS IP INSERTION ###

                                #The total expected time after adding a point to K_hold should not exceed the current slack time
                                for k in K_hold:
                                    m_insertion.addConstr(gp.quicksum(x_insertion[i][j]*t_add[i][j]*y_insertion[k][j] for i in range(len(V_move)) \

                                                                      for j in detour_pos_range) <= slack_time_rev[k])

                                #A detour position can not insert more than one stopping point
                                for j in detour_pos_range:
                                    m_insertion.addConstr(gp.quicksum(x_insertion[i][j] for i in v_move_range) <= 1)

                                # A stopping point in V_move cannot be inserted into more than one detour position
                                for i in v_move_range:
                                    m_insertion.addConstr(gp.quicksum(x_insertion[i][j] for j in detour_pos_range) <= 1)

                                # Limiting the total number of insertions 
                                if limit_to_one_insertion == 1:
                                    m_insertion.addConstr(gp.quicksum(x_insertion[i][j] for i in v_move_range for j in detour_pos_range) <= 1)
    #                             else:
    #                                 m_insertion.addConstr(gp.quicksum(x_insertion[i][j] for i in v_move_range for j in detour_pos_range) >= 1)
    #                             for i in v_move_range:
    #                                 m_insertion.addConstr(x_insertion[i][num_detour_positions-1]) == 0

                                # Calculating of total expected waiting time after adding to the resources of K_hold
                                for k in K_hold:
                                    m_insertion.addConstr(t_expupdated[k]==exp_pick_comp_time[k] +\
                                                          gp.quicksum(x_insertion[i][j]*t_add[i][j]*y_insertion[k][j] for i in v_move_range for j in detour_pos_range))

                                #Calculating of total expected waiting time after removing from a sequence of K_move
    #                             for k in K_move:
    #                                 m_insertion.addConstr(t_expupdated[k] == exp_pick_comp_time[k] - sum(t_deduct[k][i]*x_insertion[i][j] for i in v_move_range))

                                for k in K_move:
                                    m_insertion.addConstr( 
                                        t_expupdated[k] == exp_pick_comp_time[k] - 
                                        gp.quicksum(t_deduct[k][i] * x_insertion[i][j] for i in v_move_range for j in detour_pos_range)
                                    )

                                #The resources which are not part of K_move or K_hold, exp pick comp time remain unchanged
                                K_unchanged = list(set([i for i in range (num_cob + num_ded)]) - set(K_hold) -set(K_move))
                                if len(K_unchanged) != 0:
                                    for k in K_unchanged:
                                         m_insertion.addConstr(t_expupdated[k] == exp_pick_comp_time[k])

                                #Setting an upper bound for each of the updated expected travel time
                                for k in range(num_cob + num_ded):
                                    m_insertion.addConstr(z_upbound >= t_expupdated[k])

                                ### IP INSERTION - OPTIMIZATION AND RESULT ###### IP INSERTION - OPTIMIZATION AND RESULT ###### IP INSERTION - OPTIMIZATION AND RESULT ###### IP INSERTION - OPTIMIZATION AND RESULT ###
                                m_insertion.optimize()

                                selected_points_for_insertion = []
                                selected_positions_for_insertion = []
                                if m_insertion.status == GRB.OPTIMAL:
                                    print(f' updated maximum expected pick completion time: {m_insertion.ObjVal}')
                                    print(f' max pick comp time improvement: {max(exp_pick_comp_time) - m_insertion.ObjVal} seconds')
                                    insertion_improvement.append(max(exp_pick_comp_time) - m_insertion.ObjVal)
                                    for i in v_move_range:
                                          for j in detour_pos_range:
                                                if x_insertion[i][j].X > 0.001:
                                                    print(f'stopping point {V_move[i]} is inserted into detour position {j} out of {len(t_add[0])} det pos')
                                                    print(f'time added {t_add[i][j]} and deducted {dict_time_reduction_V_move[V_move[i]]}')
                                                    selected_points_for_insertion.append(V_move[i])
                                                    selected_positions_for_insertion.append(j)
                                    num_insertions_performed.append(len(selected_points_for_insertion))
                                    print(f'previous exp pick comp time: {exp_pick_comp_time}')
                                    print(f'updated exp pick comp time: {t_expupdated}')
                                else: 
                                    print('no feasible insertion found')
                                    insertion_improvement.append(0)
                                    num_insertions_performed.append(0)

                                #Need to update the sequence 
                                # First step: insert the selected points into correct sequence
    #                             pos_count = 0
                                ins_count = 0
                                for i in range(len(selected_positions_for_insertion)):
                                    pos_count = 0
                                    point = selected_points_for_insertion[i]
                                    position = selected_positions_for_insertion[i] + ins_count
                                    for k in K_hold:
                                        pos_count = pos_count + len(seq_with_current_assignments[k])-1
                                        if position < pos_count:
                                            insert_index = pos_count - position
                                            seq_with_current_assignments[k].insert(-insert_index, point)
                                            print(f' inserted {point} at index {-insert_index} of the sequence of resource {k}')
                                            break
                                    for k in K_move:
                                        if point in seq_with_current_assignments[k]:
                                            seq_with_current_assignments[k].remove(point)
                                            print(f'removed point {point} from the sequence of resource {k}')
                                    ins_count = ins_count + 1 #this is useful when there are mutliple insertions required
                                print(f'sequence with current assignments after IP insertion: {seq_with_current_assignments}')  

                                post_decision_seq_assignments_ip = seq_with_current_assignments[:]
                                print(f'seq_assignments {seq_assignments}')

    #                             for i in range (num_cob + num_ded):
    #                                 for j in range(len(seq_assignments[i])):
    #                                     if seq_assignments[i][j] == current_assignments[i]:
    #                                         break
    #                                     if seq_assignments[i][j] != post_decision_seq_assignments_ip[i][j]:
    #                                         post_decision_seq_assignments_ip[i].insert(j,seq_assignments[i][j])

                                for i in range (num_cob + num_ded):
                                    if i != nt_resource: 
                                        post_decision_seq_assignments_ip[i].insert(0,seq_assignments[i][0])
                                        print(f'added {seq_assignments[i][0]} at the beginning of sequence {i} to make the full sequence')

    #                             prev_point_of_picked_point_index = post_decision_seq_assignments_ip[nt_resource].index(nt_point) - 1
    #                             print(f'removed point {post_decision_seq_assignments_ip[nt_resource][prev_point_of_picked_point_index]} \
    #                             from the sequence of resource {nt_resource} as all items have been picked')
    #                             post_decision_seq_assignments_ip[nt_resource].remove(post_decision_seq_assignments_ip[nt_resource][prev_point_of_picked_point_index])
                                seq_assignments = post_decision_seq_assignments_ip[:] 
                                print(f'finalized post decision sequence after IP insertion: {seq_assignments}')

                                print(f'------------------------------------- END: IP INSERTION -------------------------------------------')
                                #-----------------------------------------------------------------------------------------------------------------------------------------------------------
                                # END: IP Insertion
                                #-----------------------------------------------------------------------------------------------------------------------------------------------------------


                            elif run_ipinsertion == 0:
                                num_insertions_performed.append(0)
                                insertion_improvement.append(0)
                                prev_point_of_picked_point_index = seq_assignments[nt_resource].index(nt_point) - 1
                                print(f'removed point {seq_assignments[nt_resource][prev_point_of_picked_point_index]} \
                                from the sequence of resource {nt_resource} as all items have been picked')
                                seq_assignments[nt_resource].remove(seq_assignments[nt_resource][prev_point_of_picked_point_index])

                        #Condition if there is no sequence having exp pick comp time greater than the wave, this only gets activated when we choose to set update_when_infeasible_routes is set to 1
                        if greater_than_wave == 0:
                            print('we do not update the sequences as all resources have exp pick comp time less than the wave')
                            prev_point_of_picked_point_index = seq_assignments[nt_resource].index(nt_point) - 1
                            print(f'removed point {seq_assignments[nt_resource][prev_point_of_picked_point_index]} \
                            from the sequence of resource {nt_resource} as all items have been picked')
                            seq_assignments[nt_resource].remove(seq_assignments[nt_resource][prev_point_of_picked_point_index])


                        #Update current assignments
                        current_assignments[nt_resource] = seq_assignments[nt_resource][1]
                        print(f'updated current assignments:{current_assignments}')

                        #Condition if the current assignment is not feasible before the return trigger
                        if nt_resource < num_cob:
                            min_feasible_time = nt_time + time_mat_req_cob.loc[seq_assignments[nt_resource][0], current_assignments[nt_resource]] +\
                            time_mat_req_cob.loc[current_assignments[nt_resource], v_p] + avg_pick_time
                        else:
                            min_feasible_time = nt_time + time_mat_req_ded.loc[seq_assignments[nt_resource][0], current_assignments[nt_resource]] +\
                            time_mat_req_ded.loc[current_assignments[nt_resource], v_p] + avg_pick_time

                        if min_feasible_time >=t_wave:
                            print(f'it is feasible to travel to the dropoff station instead of the next assignment')
                            current_assignments[nt_resource] = v_p
                            #drop_off_station_reach_time_list[nt_resource] = nt_time + time_mat_req_ded.loc[seq_assignments[nt_resource][0], current_assignments[nt_resource]]

                        # Update reach time, sync time ann pick completion time lists
                        if nt_resource < num_cob:
                            reach_time_list[nt_resource] = nt_time + time_mat_req_cob.loc[seq_assignments[nt_resource][0], current_assignments[nt_resource]]
                            cobot_travel_time.append(time_mat_req_cob.loc[seq_assignments[nt_resource][0], current_assignments[nt_resource]])
                        else:
                            reach_time_list[nt_resource] = nt_time + time_mat_req_ded.loc[seq_assignments[nt_resource][0], current_assignments[nt_resource]]
                            dedicated_travel_time.append(time_mat_req_ded.loc[seq_assignments[nt_resource][0], current_assignments[nt_resource]])
                        print(f'updated reach time:{reach_time_list}')


                        #Update the sync_time list
                        if current_assignments[nt_resource] == v_p: #Condition if the next assignment is the packing station
                            sync_time[nt_resource] = t_wave
                            drop_off_station_reach_time_list[nt_resource] = reach_time_list[nt_resource] # Recording the reaching time to dropoff station
                        elif nt_resource<num_cob:
                            for j in range(arr_mat.shape[0]):
                                if arr_mat.iloc[j, current_assignments[nt_resource] -1] + dict_dwell[current_assignments[nt_resource]]>=\
                                reach_time_list[nt_resource]:
                                    if reach_time_list[nt_resource] <= arr_mat.iloc[j, current_assignments[nt_resource] -1]:
                                        sync_time[nt_resource] = arr_mat.iloc[j, current_assignments[nt_resource] -1]
                                    else: 
                                        sync_time[nt_resource] = reach_time_list[nt_resource]
                                    break
                        else:
                            sync_time[nt_resource] =  reach_time_list[nt_resource]
                        print(f' updated sync time list:{sync_time}')

                        #Update pick completion trigger list
                        if current_assignments[nt_resource] == v_p:
                            pick_completion_trigger[nt_resource] = t_wave
                        else:
                            pick_comp_next_assgn = avg_pick_time*dict_num_picks[current_assignments[nt_resource]]
    #                         pick_comp_next_assgn = avg_pick_time*\
    #                         (1+((dict_num_picks[current_assignments[nt_resource]]-1)*pick_time_shortening_factor))
                            pick_completion_trigger[nt_resource] = sync_time[nt_resource] + pick_comp_next_assgn
                        print(f' updated pick completion trigger list:{pick_completion_trigger}')

    #                     print(f'trigger return: {trigger_return})
                        # Update return trigger list
                        if current_assignments[nt_resource] == v_p:
                            trigger_return[nt_resource] = t_wave
                        else:
                            if nt_resource < num_cob:
                                trigger_return[nt_resource] = t_wave - time_mat_req_cob.loc[current_assignments[nt_resource], v_p]
                            else:
                                trigger_return[nt_resource] = t_wave - time_mat_req_ded.loc[current_assignments[nt_resource], v_p]
                        print(f' updated return trigger:{trigger_return}')

                        #Update abandon trigger list
                        if use_abandon ==1:
                            if current_assignments[nt_resource] != v_p:
                                if nt_resource<num_cob:
                                    updtd_abndn = abaondon_policy (seq_assignments[nt_resource], dict_buy_rate, n_picks_list, nt_time,\
                                                                     time_mat_req_cob, dict_exp_wait_cobot)
                                    trigger_abandon[nt_resource] = reach_time_list[nt_resource] + updtd_abndn
                                else:
                                    trigger_abandon[nt_resource] = t_wave
                            elif current_assignments[nt_resource] == v_p:
                                trigger_abandon[nt_resource] = t_wave

                        #Updating next trigger info
                        nt_time, nt_type, nt_resource, nt_point = next_trigger (pick_completion_trigger, trigger_return, trigger_abandon) #Here nt refers to next trigger
                        print(f' next trigger time: {nt_time}, trigger type: {nt_type}, by resource:{nt_resource}, at point :{nt_point}')
                        pick_completion_end_time = time.time()
                        pick_completion_trigger_time.append(pick_completion_end_time - pick_completion_start_time)

                # TRIGGER TYPE - RETURN TRIGGER ---------------------TRIGGER TYPE - RETURN TRIGGER----------------------TRIGGER TYPE - RETURN TRIGGER-------------------------  
                elif nt_type == 'return':

                    return_trigger_start_time = time.time()
                    # Recording the reaching time of resources to the drop-off station                                    
                    if nt_resource < num_cob:
                        drop_off_station_reach_time_list[nt_resource] = nt_time +  time_mat_req_cob.loc[current_assignments[nt_resource],v_p]
                        cobot_travel_time.append(time_mat_req_cob.loc[current_assignments[nt_resource],v_p])
                    else:
                        drop_off_station_reach_time_list[nt_resource] = nt_time +  time_mat_req_ded.loc[current_assignments[nt_resource],v_p]
                        dedicated_travel_time.append(time_mat_req_ded.loc[current_assignments[nt_resource],v_p])

                    #Calculate the amount of items picked
                    if nt_time <= sync_time[nt_resource]:
                        partially_picked = 0
                        if nt_resource < num_cob:
                            cobot_wait_time.append(nt_time - reach_time_list[nt_resource])

                    elif nt_time > sync_time[nt_resource]:
                        if nt_time - sync_time[nt_resource] - avg_pick_time >=0:
                            partially_picked = 1+(nt_time - sync_time[nt_resource] - avg_pick_time)//(avg_pick_time*pick_time_shortening_factor)
                            #Recording the cobot and dedicated wait and pick time
                            if nt_resource < num_cob:
                                cobot_wait_time.append(sync_time[nt_resource] - reach_time_list[nt_resource])
                                cobot_pick_time.append(partially_picked*avg_pick_time)
                                cobot_incomplete_pick_time.append(nt_time - sync_time[nt_resource] - (partially_picked*avg_pick_time))
                            else:
                                dedicated_pick_time.append(partially_picked*avg_pick_time)
                                dedicated_incomplete_pick_time.append(nt_time - sync_time[nt_resource] - (partially_picked*avg_pick_time))
                        else:
                            partially_picked = 0
                            #Recording the cobot and dedicated wait and pick time
                            if nt_resource < num_cob:
                                cobot_wait_time.append(sync_time[nt_resource] - reach_time_list[nt_resource])
                                cobot_pick_time.append(partially_picked*avg_pick_time)
                                cobot_incomplete_pick_time.append(nt_time - sync_time[nt_resource])
                            else:
                                dedicated_pick_time.append(partially_picked*avg_pick_time)
                                dedicated_incomplete_pick_time.append(nt_time - sync_time[nt_resource])


                    # Record parial pickings if any
                    if nt_resource < num_cob:
                        partial_aisles_by_cobot.append(nt_point)
                        num_picked_by_cobot.append(partially_picked)
                        print(f'# of items partially picked by cobot {partially_picked}')
                    else:
                        partial_aisles_by_dedicated.append(nt_point)
                        num_picked_by_dedicated.append(partially_picked)
                        print(f'# of items partially picked by dedicated {partially_picked}')

                    #Update trigger information
                    current_assignments[nt_resource] = v_p
                    reach_time_list[nt_resource] = t_wave
                    pick_completion_trigger[nt_resource] = t_wave
                    trigger_return[nt_resource] = t_wave
                    trigger_abandon[nt_resource] = t_wave

                    #Update next trigger information
                    nt_time, nt_type, nt_resource, nt_point = next_trigger (pick_completion_trigger, trigger_return, trigger_abandon) #Here nt refers to next trigger
                    print(f' next trigger time: {nt_time}, trigger type: {nt_type}, by resource:{nt_resource}, at point :{nt_point}')

                    return_trigger_end_time = time.time()
                    print(f'return trigger computational time {return_trigger_end_time - return_trigger_start_time}')
                    return_trigger_time.append(return_trigger_end_time - return_trigger_start_time)
                  #Updating decisions if the trigger is of abandon type   


                # TRIGGER TYPE - ABANDON TRIGGER ---------------------TRIGGER TYPE - ABANDON TRIGGER----------------------TRIGGER TYPE - ABANDON TRIGGER------
                elif nt_type == 'abandon':
                    #Recond the number of abandoning trigger events
                    abandon_trigger_start_time = time.time()
                    abandon_events.append(1)

                    #re-calculate abandon time for the current point
                    print(f'sequence of assignments before abandon decision: {seq_assignments}')
                    updtd_abndn = abaondon_policy (seq_assignments[nt_resource], dict_buy_rate, n_picks_list, nt_time,\
                                                                 time_mat_req_cob, dict_exp_wait_cobot)

                    if nt_time >= sync_time[nt_resource]:
                        print('invalid abandon trigger as synch has happened before abandon trigger')
                        invalid_abandon_trigger.append(1)
                        trigger_abandon[nt_resource] = nt_time + dict_num_picks[nt_point]*avg_pick_time
                    elif updtd_abndn >1:
                        print(f'decision is to wait {updtd_abndn} more seconds')
                        trigger_abandon[nt_resource] = nt_time + updtd_abndn
                    elif updtd_abndn <1 and len(seq_assignments[nt_resource])<=3:
                        print(f'abandoning does not make sense, as {nt_point} is the last point in the sequence: {nt_resource}')
                        trigger_abandon[nt_resource] = t_wave
                    else:
                        print(f'decision is to abandon point {nt_point}')
                        abandon_performance.append([nt_resource, nt_point]) # Recording which resource abandoned which point
                        cobot_wait_time.append(nt_time-reach_time_list[nt_resource])
                        abandoned_point = nt_point #redundant as abandonded point is the nt point
                        print(f' deleted point {seq_assignments[nt_resource][0]} from sequence {nt_resource}')
                        del(seq_assignments[nt_resource][0]) #Deleting the point before the current assignment
                        current_assignments[nt_resource] = seq_assignments[nt_resource][1]



                        #Need to decide at the end of which sequence the abandoned point will be added
                        #First step - create sequences without their stations
                        seq_wo_stations_abandon = []
                        for i in range(num_cob + num_ded):
                            seq_wo_stations_abandon.append(seq_assignments[i][1:-1])
                        print(f'sequence without stations{seq_wo_stations_abandon}')

                        #Second step - calculate expected pick completion time for current sequences
                        exp_pick_comp_time_abandon = []

                        for i in range(num_cob + num_ded):
                            if i < num_cob:
                                exp_wait_time_abandon = [dict_exp_wait_cobot[j] for j in seq_wo_stations_abandon[i]]
                            else:
                                exp_wait_time_abandon = [0 for j in seq_wo_stations_abandon[i]] #this is redundant as dedicated pickers do not face any abandon triggers
                            exp_pick_time_abandon = [avg_pick_time + (dict_num_picks[j]-1)*pick_time_shortening_factor*avg_pick_time for j in seq_wo_stations[i]]
                            travel_time_abandon = []
                            seq_for_travel_time_calc = seq_wo_stations_abandon[i]+[v_p]
                            if i < num_cob:
                                for k in range(len(seq_for_travel_time_calc)-1):
                                    travel_time_abandon.append(time_mat_req_cob.loc[seq_for_travel_time_calc[k],seq_for_travel_time_calc[k+1]])
                            else:
                                for k in range(len(seq_for_travel_time_calc)-1):
                                    travel_time_abandon.append(time_mat_req_ded.loc[seq_for_travel_time_calc[k],seq_for_travel_time_calc[k+1]])

                            if current_assignments[i] == v_p:
                                exp_pick_comp_time_abandon.append(10000)

                            elif nt_time < sync_time[i]:
            #                 if nt_time < reach_time_list[i] or (nt_time > reach_time_list[i] and nt_time < sync_time[i]):
                                print(f'trigger time {nt_time} of resource {nt_resource} is earlier than reach time of resource {i} or earlier than sync time of {sync_time[i]}')
                                exp_pick_comp_time_abandon.append(sum(exp_wait_time_abandon) + sum(exp_pick_time_abandon) + sum(travel_time_abandon) + reach_time_list[i])

                            elif nt_time > sync_time[i] and nt_time < pick_completion_trigger[i]:          
                                print(f'trigger time: {nt_time}s of resource {nt_resource} is later than sync time of {sync_time[i]} of resource {i}')
                                exp_pick_comp_time_abandon.append(sum(exp_wait_time_abandon) + sum(exp_pick_time_abandon ) + sum(travel_time_abandon ) \
                                                          + nt_time - exp_wait_time_abandon[0])

                            elif nt_time == pick_completion_trigger[i]:
                                print(f'pick completion calculation for the triggered resource # {nt_resource}')
                                exp_pick_comp_time_abandon.append(sum(exp_wait_time_abandon ) + sum(exp_pick_time_abandon ) + sum(travel_time_abandon) \
                                                          + nt_time - exp_wait_time_abandon[0] - exp_pick_time_abandon[0])

                        print(f'expected pick completion time list in abandon trigger {exp_pick_comp_time_abandon}')

                        #Third step - identify which sequence can add the abandoned point to the end of their sequence and update the finalized sequence
                        temp_seqs = seq_assignments[:]
                        max_exp_pick_comp_list = 0
                        inserted_exp_pick_comp_time = []
                        for i in range(num_cob+num_ded):
                            if i < num_cob:
                                inserted_exp_pick_comp_time.append(exp_pick_comp_time_abandon[i] + dict_exp_wait_cobot[abandoned_point] + avg_pick_time +\
                                                                  (dict_num_picks[abandoned_point]-1)*pick_time_shortening_factor*avg_pick_time +\
                                                                  time_mat_req_cob.loc[seq_assignments[i][-2],abandoned_point] + time_mat_req_cob.loc[abandoned_point, seq_assignments[i][-1]]\
                                                                  - time_mat_req_cob.loc[seq_assignments[i][-2],seq_assignments[i][-2]])
                            else:
                                inserted_exp_pick_comp_time.append(avg_pick_time + exp_pick_comp_time_abandon[i]+\
                                              (dict_num_picks[abandoned_point]-1)*pick_time_shortening_factor*avg_pick_time +\
                                              time_mat_req_ded.loc[seq_assignments[i][-2],abandoned_point] + time_mat_req_ded.loc[abandoned_point, seq_assignments[i][-1]]\
                                              - time_mat_req_ded.loc[seq_assignments[i][-2],seq_assignments[i][-2]])

                        print(f'exp pick time list after calculating insertion of abandoned point:{inserted_exp_pick_comp_time}')
                        temp_ins = exp_pick_comp_time_abandon[:]
                        temp_max_calc = []
                        for i in range(num_cob + num_ded):
                            temp_ins[i] =  inserted_exp_pick_comp_time[i]
                            temp_max_calc.append(max(temp_ins))
                            temp_ins = exp_pick_comp_time_abandon[:]

                        print(f'max expected pick time after inserting into resources list: {temp_max_calc }')
                        max_increase_list = []
                        for i in range(num_cob + num_ded):
                            max_increase_list.append(temp_max_calc[i] - max(exp_pick_comp_time_abandon))
                            print(f' max exp pick time increases {temp_max_calc[i] - max(exp_pick_comp_time_abandon)} after inserting into resource {i}')
                        abandoned_point_insertion_resource = max_increase_list.index(min(max_increase_list))
                        print(f'decision is to add abandoned point {abandoned_point} into sequence of resource {abandoned_point_insertion_resource}')
                        seq_assignments[abandoned_point_insertion_resource].insert(-1,abandoned_point) #Add the abandoned point
                        #seq_assignments[nt_resource].remove(nt_point) #Remove the point from the previous sequence


                        #Update xi, the trigger information tuple
                        if nt_resource < num_cob:
                            reach_time_list[nt_resource] = nt_time + time_mat_req_cob.loc[nt_point, current_assignments[nt_resource]]
                            cobot_travel_time.append(time_mat_req_cob.loc[nt_point, current_assignments[nt_resource]])
                        else: # This is redundant as dedicated pickers do not face any abandon triggers
                            reach_time_list[nt_resource] = nt_time + time_mat_req_ded.loc[nt_point, current_assignments[nt_resource]]
                        print(f'updated reach time list after abandoning {reach_time_list}')

                        for j in range(arr_mat.shape[0]):
                            if arr_mat.iloc[j, current_assignments[nt_resource] -1] + dict_dwell[current_assignments[nt_resource]]\
                            >=reach_time_list[nt_resource]:
                                if reach_time_list[nt_resource] <= arr_mat.iloc[j, current_assignments[nt_resource] -1]:
                                    sync_time[nt_resource] = arr_mat.iloc[j, current_assignments[nt_resource] -1]
                                    print(f'here 1 arrival {arr_mat.iloc[j, current_assignments[nt_resource] -1]} and dwell {dict_dwell[current_assignments[nt_resource]]}')
                                else:
                                    sync_time[nt_resource] = reach_time_list[nt_resource]
                                break    
                        print(f'updated sync time of resource {nt_resource} after abandoning {sync_time[nt_resource]}')

                        if current_assignments[nt_resource] != v_p:
    #                         pick_completion_trigger[nt_resource] = (avg_pick_time *(1 + (dict_num_picks[current_assignments[nt_resource]]-1)*\
    #                                                                                pick_time_shortening_factor)) + sync_time[nt_resource]
                            pick_completion_trigger[nt_resource] = (avg_pick_time*dict_num_picks[current_assignments[nt_resource]]) + sync_time[nt_resource]
                        else:
                            pick_completion_trigger[nt_resource] = t_wave

                        if nt_resource < num_cob:
                            trigger_return[nt_resource] = t_wave - time_mat_req_cob.loc[current_assignments[nt_resource],v_p]
                        else:
                            trigger_return[nt_resource] = t_wave - time_mat_req_ded.loc[current_assignments[nt_resource],v_p]


                        if len(seq_assignments[nt_resource]) > 2: 
                            trigger_abandon[nt_resource] = reach_time_list[nt_resource] + abaondon_policy (seq_assignments[nt_resource], dict_buy_rate, n_picks_list, nt_time,\
                                                                     time_mat_req_cob, dict_exp_wait_cobot)
                        else:
                            trigger_abandon[nt_resource] = t_wave

                    #Determine the next trigger
                    nt_time, nt_type, nt_resource, nt_point = next_trigger (pick_completion_trigger, trigger_return, trigger_abandon) #Here nt refers to next trigger
                    print(f' after facing abandon trigger, next trigger time: {nt_time}, trigger type: {nt_type}, by resource:{nt_resource}, at point :{nt_point}')
                    abandon_trigger_end_time = time.time()
                    abandon_trigger_time.append(abandon_trigger_end_time-abandon_trigger_start_time)

                print(f'-----------------------------------------end of trigger: {trig_num}-------------------------------------------------------------------')
                trig_num = trig_num + 1

                if current_assignments == termination_crtieria:
                    if nt_time != t_wave:
                        if nt_resource < num_cob:
                            print(f'all resources at the dropoff station at {nt_time+time_mat_req_cob.loc[nt_point, v_p]}s')
                        else:
                            print(f'all resources at the dropoff station at {nt_time+time_mat_req_ded.loc[nt_point, v_p]}s')
                    else:
                        print(f'all resources at the dropoff station at {t_wave}s')

            print('-----------------------------------------------------------------------------------------------------------------------------------------------------------')
            print(f'--------------------------------------------------------- END : order scenario {ord_scen}-----------------------------------------------------------------')
            print('-----------------------------------------------------------------------------------------------------------------------------------------------------------')
            #-------------------------------------------------------------------------------------------------------------------------------
            # REPORTING BLOCK (WAVE) ##REPORTING BLOCK (WAVE)REPORTING BLOCK (WAVE)REPORTING BLOCK (WAVE)REPORTING BLOCK (WAVE)
            #------------------------------------------------------------------------------------------------------------------------------
            total_aisles_by_cobot.append(len(aisles_by_cobot))
            total_aisles_by_dedicated.append(len(aisles_by_dedicated))
            total_picked_by_cobot.append(sum(num_picked_by_cobot)) 
            total_picked_by_dedicated.append(sum(num_picked_by_dedicated))
            list_partial_picked_by_cobot.append(partial_aisles_by_cobot)
            list_partial_picked_by_dedicated.append(partial_aisles_by_dedicated)
            if nt_time != t_wave:
                if nt_resource < num_cob:
                    pick_end_time.append(nt_time+time_mat_req_cob.loc[nt_point, v_p])
                else:
                    pick_end_time.append(nt_time+time_mat_req_ded.loc[nt_point, v_p])
            else:
                pick_end_time.append(t_wave)
            # report_items_in_order.append(n_picks)
            # report_aisles_in_order.append(count())

            if ranking_based_seq == 1:
                sequencing_alternative.append('buy')
            elif buypicks_based_seq == 1: 
                sequencing_alternative.append('R')
            else:
                sequencing_alternative.append('T')
                
            if use_abandon == 1:  
                abandon_alternative.append('A')
            else:
                abandon_alternative.append('N')

            if use_rsia == 1:
                updating_alternative.append('rsia')
            else:
                updating_alternative.append('ip')

            limiting_alternative.append(limit_to_one_insertion)
            
            
            if no_update == 1:
                report_limiting_alternative.append('Z')
            elif limit_to_one_insertion == 1:
                report_limiting_alternative.append('S')
            elif limit_to_one_insertion == 0:
                report_limiting_alternative.append('M')


            # Report utilization of resources
            report_cobot_travel_time.append(sum(cobot_travel_time))
            report_cobot_pick_time.append(sum(cobot_pick_time))
            report_cobot_wait_time.append(sum(cobot_wait_time))
            report_dedicated_travel_time.append(sum(dedicated_travel_time))
            report_dedicated_pick_time.append(sum(dedicated_pick_time))


            #Record the incomplete picking times caused by the return trigger
            report_cobot_incomplete_pick_time.append(sum(cobot_incomplete_pick_time))
            report_dedicated_incomplete_pick_time.append(sum(dedicated_incomplete_pick_time))

            #Record the idle time at the end of a resource's picking journey
            report_cobot_idle_time.append((num_cob*t_wave) - sum(cobot_travel_time) - sum(cobot_pick_time) - sum(cobot_wait_time) - sum(cobot_incomplete_pick_time))
            report_dedicated_idle_time.append((num_ded*t_wave) - sum(dedicated_travel_time) - sum(dedicated_pick_time) - sum(dedicated_incomplete_pick_time))


            #Report drop off station reach time list
            report_drop_off_station_reach_time_list.append(drop_off_station_reach_time_list)

            # Report abandoning performance
            report_abandon_performance.append(abandon_performance)
            report_actually_abandoned.append(len(abandon_performance))
            report_abandon_triggering_events.append(sum(abandon_events) - sum(invalid_abandon_trigger))


            #Report Computational Time
            report_clustering_time.append(clustering_time)
            report_sequencing_time.append(sequencing_time)
            report_pick_completion_trigger_time.append(pick_completion_trigger_time)
            report_abandon_trigger_time.append(abandon_trigger_time)
            report_return_trigger_time.append(return_trigger_time)
            report_participating_cust.append(part_cust_num)
            report_update_when_infeasible.append(update_when_infeasible_routes)
            
            # Report resource levels
            report_cobots.append(num_cob)
            report_dedicateds.append(num_ded)
            
            # Rerport Customer Arrival Instance Number
            report_custArrInst.append(arr_count+1)

#Final Report 
dict_report = {
    'custArrInst' : report_custArrInst,
    'numItems': report_items_in_order,
    'numAisles': report_aisles_in_order,
    'numPickedCobot': total_picked_by_cobot,
    'numPickedDedicated': total_picked_by_dedicated,
    'numAislesCobot': total_aisles_by_cobot,
    'numAislesDedicated': total_aisles_by_dedicated,
    'partialAislesCobot': list_partial_picked_by_cobot,
    'partialAislesDedicated': list_partial_picked_by_dedicated,
    'sequenceVariant': sequencing_alternative,
    'abandonVariant': abandon_alternative,
    'updating_alternative': updating_alternative,
    'limiting_alternative': report_limiting_alternative,
    'updateWhenInfeasible': report_update_when_infeasible,
    'pickEndTime': pick_end_time,
    'cobotTravelTime': report_cobot_travel_time,
    'cobotPickTime': report_cobot_pick_time,
    'cobotIncompletePickTime': report_cobot_incomplete_pick_time,
    'cobotWaitTime': report_cobot_wait_time,
    'cobotIdleTime': report_cobot_idle_time,
    'dedicatedTravelTime': report_dedicated_travel_time,
    'dedicatedPickTime': report_dedicated_pick_time,
    'dedicatedIncompletePickTime': report_dedicated_incomplete_pick_time,
    'dedicatedIdleTime': report_dedicated_idle_time,
    'dropoffReachTime': report_drop_off_station_reach_time_list,
    'num_cob': report_cobots,
    'num_ded': report_dedicateds,
    'abandonDetails': report_abandon_performance,
    'abandonActual': report_actually_abandoned,
    'abandonTrigger': report_abandon_triggering_events,
    'clusteringTime': report_clustering_time,
    'sequencingTime': report_sequencing_time,
    'pickCompTrigTime': report_pick_completion_trigger_time,
    'abandonTrigTime': report_abandon_trigger_time,
    'returnTrigTime': report_return_trigger_time,
    'participatingCustomers': report_participating_cust
}
df_report = pd.DataFrame.from_dict(dict_report)
df_report.head()
df_report.to_excel('C6D1-RAS.xlsx')

# Restore sys.stdout to its original value
sys.stdout = original_stdout

# Close the file
file.close()